In [1]:
import pandas as pd
import numpy as np

## 数据清洗

#### 0/1型字符数据转换为数值

In [2]:

#导入数据+把0/1型字符数据转换为数值
data_path='data.xlsx'
data=pd.read_excel(io=data_path,encoding='gbk')
def transform(row):
    if row['是否续保']=='是':
        row['是否续保']='1'
    else:
        row['是否续保']='0'
    return row
def transform2(row):
    if row['客户类别']=='个人':
        row['客户类别']='1'
    else:
        row['客户类别']='0'
    return row
def transform3(row):
    if row['是否投保车损']=='投保车损':
        row['是否投保车损']='1'
    else:
        row['是否投保车损']='0'
    return row
def transform4(row):
    if row['是否投保盗抢']=='投保车损':
        row['是否投保盗抢']='1'
    else:
        row['是否投保盗抢']='0'
    return row
def transform5(row):
    if row['是否投保车上人员']=='投保车损':
        row['是否投保车上人员']='1'
    else:
        row['是否投保车上人员']='0'
    return row
def transform6(row):
    if row['保单性质']=='转保':
        row['保单性质']='1'
    else:
        row['保单性质']='0'
    return row
def transform7(row):
    if row['客户类别']=='个人':
        row['客户类别']='1'
    else:
        row['客户类别']='0'
    return row
def transform8(row):
    if row['险种']=='交强险':
        row['险种']='1'
    else:
        row['险种']='0'
    return row







#### 删除无用项

In [ ]:
#删除部分数据
data=data.drop(['品牌'],axis=1)
data=data.drop(['车系'],axis=1)
data=data.drop(['起保日期'],axis=1)
data=data.drop(['终止日期'],axis=1)
data=data.drop(['保单号'],axis=1)
data=data[~data['NCD'].isin(['新保无记录'])]
data=data[~data['车辆用途'].isin(['重、中型载货汽车'])]
data=data[~data['车辆用途'].isin(['其他汽车'])]
data=data[~data['车辆种类'].isin(['特种二挂车'])]


In [ ]:


#生成训练用数据
result=np.array(data['是否续保'].values)
data=data.drop(['是否续保'],axis=1)
predict=np.array(data.values)
predict1=predict[:50000]
predict2=predict[50000:-1]
result1=result[:50000]
result2=result[50000:-1]

#训练logistics回归模型
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=100000,tol=1e-7)
lr.fit(predict1,result1)


#数据归一化
def zero_one1(row):
    row['三者险保额']=row['三者险保额']/2000000
    return row
def zero_one2(row):
    row['签单保费']=row['签单保费']/38297
    return row
def zero_one3(row):
    row['立案件数']=row['立案件数']/2000000
    return row
def zero_one4(row):
    row['已决赔款']=row['已决赔款']/983620.00
    return row
def zero_one5(row):
    row['新车购置价']=row['新车购置价']/5100000
    return row
def zero_one6(row):
    row['车龄']=row['车龄']/24
    return row
def zero_one7(row):
    row['被保险人年龄']=row['被保险人年龄']/75
    return row
def zero_one8(row):
    row['续保年']=row['续保年']/8
    return row

data=data.apply(zero_one1,axis=1)
data=data.apply(zero_one2,axis=1)
data=data.apply(zero_one3,axis=1)
data=data.apply(zero_one4,axis=1)
data=data.apply(zero_one5,axis=1)
data=data.apply(zero_one6,axis=1)
data=data.apply(zero_one7,axis=1)
data=data.apply(zero_one8,axis=1)


#训练神经网络
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='relu',solver='adam',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.01,max_iter=2000)
model.fit(predict1,result1)
acc=model.score(predict2,result2) #根据给定数据与标签返回正确率的均值
print('神经网络模型评价:',acc)

#### 填充NA数据

In [ ]:

#将部分数据NAN填充
data['被保险人性别']=data['被保险人性别'].fillna('N')
data['已决赔款']=data['已决赔款'].fillna(0)
data['三者险保额']=data['三者险保额'].fillna(0)
data=data.fillna(0)

#### 利用onehot把字符型数据转换为数字类型

In [5]:
#利用onehot把值转换为数字类型
data_dummies1=pd.get_dummies(data['车辆种类'])
data_dummies2=pd.get_dummies(data['渠道'])
data_dummies3=pd.get_dummies(data['投保类别'])
data_dummies4=pd.get_dummies(data['车辆用途'])
data_dummies5=pd.get_dummies(data['是否本省车牌'])
data_dummies6=pd.get_dummies(data['NCD'])
data_dummies7=pd.get_dummies(data['风险类别（A最低，E最高）'])
data_dummies8=pd.get_dummies(data['使用性质'])
data_dummies9=pd.get_dummies(data['被保险人性别'])

data=data.drop(['车辆种类'],axis=1)
data=data.drop(['渠道'],axis=1)
data=data.drop(['投保类别'],axis=1)
data=data.drop(['车辆用途'],axis=1)
data=data.drop(['是否本省车牌'],axis=1)
data=data.drop(['NCD'],axis=1)
data=data.drop(['风险类别（A最低，E最高）'],axis=1)
data=data.drop(['使用性质'],axis=1)
data=data.drop(['客户类别'],axis=1)
data=data.drop(['被保险人性别'],axis=1)

data=pd.concat([data,data_dummies1],axis=1) 
data=pd.concat([data,data_dummies2],axis=1) 
data=pd.concat([data,data_dummies3],axis=1) 
data=pd.concat([data,data_dummies4],axis=1) 
data=pd.concat([data,data_dummies5],axis=1) 
data=pd.concat([data,data_dummies6],axis=1) 
data=pd.concat([data,data_dummies7],axis=1) 
data=pd.concat([data,data_dummies8],axis=1) 
data=pd.concat([data,data_dummies9],axis=1) 

In [7]:
#生成训练用数据
result=np.array(data['是否续保'].values)
data=data.drop(['是否续保'],axis=1)
predict=np.array(data.values)
predict1=predict[:50000]
predict2=predict[50000:-1]
result1=result[:50000]
result2=result[50000:-1]

#训练logistics回归模型
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=100000,tol=1e-7)
lr.fit(predict1,result1)


#数据归一化
def zero_one1(row):
    row['三者险保额']=row['三者险保额']/2000000
    return row
def zero_one2(row):
    row['签单保费']=row['签单保费']/38297
    return row
def zero_one3(row):
    row['立案件数']=row['立案件数']/2000000
    return row
def zero_one4(row):
    row['已决赔款']=row['已决赔款']/983620.00
    return row
def zero_one5(row):
    row['新车购置价']=row['新车购置价']/5100000
    return row
def zero_one6(row):
    row['车龄']=row['车龄']/24
    return row
def zero_one7(row):
    row['被保险人年龄']=row['被保险人年龄']/75
    return row
def zero_one8(row):
    row['续保年']=row['续保年']/8
    return row

data=data.apply(zero_one1,axis=1)
data=data.apply(zero_one2,axis=1)
data=data.apply(zero_one3,axis=1)
data=data.apply(zero_one4,axis=1)
data=data.apply(zero_one5,axis=1)
data=data.apply(zero_one6,axis=1)
data=data.apply(zero_one7,axis=1)
data=data.apply(zero_one8,axis=1)


#训练神经网络
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='relu',solver='adam',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.01,max_iter=2000)
model.fit(predict1,result1)
acc=model.score(predict2,result2) #根据给定数据与标签返回正确率的均值
print('神经网络模型评价:',acc)

NameError: name 'predict1' is not defined

In [7]:
result=np.array(data['是否续保'].values)
data=data.drop(['是否续保'],axis=1)
predict=np.array(data.values)

In [9]:
predict1=predict[:50000]
predict2=predict[50000:-1]
result1=result[:50000]
result2=result[50000:-1]

In [1]:
from sklearn.externals import joblib
#joblib.dump(lr,"logistic_lr.model")  
a=joblib.load("logistic_lr.model")

In [20]:
data.iloc[3:5,64:65]

,5.上年发生2+有责事故
3,0
4,0


In [53]:
c=b[0]

In [54]:
a.coef_

array([[-1.14539253e+01, -9.42519578e-01, -9.42519578e-01,
        -4.41713772e-01, -4.41713772e-01, -4.32707589e-01,
        -4.32707589e-01, -4.30917426e-01, -4.30917426e-01,
        -4.02114658e-01, -4.02114658e-01, -2.63143897e-01,
        -2.38901209e-01, -2.38901209e-01, -2.33167925e-01,
        -2.33167925e-01, -1.93825452e-01, -1.93825452e-01,
        -1.87115031e-01, -1.87115031e-01, -1.42009181e-01,
        -1.42009181e-01, -1.39693889e-01, -1.39693889e-01,
        -1.32837027e-01, -1.32837027e-01, -1.31300719e-01,
        -1.31300719e-01, -1.29875987e-01, -1.29875987e-01,
        -1.12760667e-01, -1.12760667e-01, -1.09967386e-01,
        -1.09967386e-01, -1.06610979e-01, -1.06610979e-01,
        -8.63607630e-02, -8.63607630e-02, -7.81197822e-02,
        -7.70194576e-02, -4.68056327e-02, -4.68056327e-02,
        -1.46069908e-02, -1.46069908e-02, -1.28099507e-02,
        -1.28099507e-02, -6.24965099e-03, -6.24965099e-03,
        -2.31300671e-03, -2.31300671e-03, -2.00459735e-0

In [22]:
def search(n,list):
    i=0
    for i in range(0,173):
        if list[i]==n:
            return i
    return -1

search(0.644356684,a.coef_[0])


-1

NameError: name 'data' is not defined

In [1]:
from sklearn.svm import SVC
clf=SVC(probability=True,tol=1e-3)
clf.fit(predict1,result1)

NameError: name 'predict1' is not defined

In [9]:
data.to_csv('temp.csv')
data=pd.read_csv('temp.csv',index_col=0)

In [14]:
#数据归一化
def zero_one1(row):
    row['三者险保额']=row['三者险保额']/2000000
    return row
def zero_one2(row):
    row['签单保费']=row['签单保费']/38297
    return row
def zero_one3(row):
    row['立案件数']=row['立案件数']/2000000
    return row
def zero_one4(row):
    row['已决赔款']=row['已决赔款']/983620.00
    return row
def zero_one5(row):
    row['新车购置价']=row['新车购置价']/5100000
    return row
def zero_one6(row):
    row['车龄']=row['车龄']/24
    return row
def zero_one7(row):
    row['被保险人年龄']=row['被保险人年龄']/75
    return row
def zero_one8(row):
    row['续保年']=row['续保年']/8
    return row

data=data.apply(zero_one1,axis=1)
data=data.apply(zero_one2,axis=1)
data=data.apply(zero_one3,axis=1)
data=data.apply(zero_one4,axis=1)
data=data.apply(zero_one5,axis=1)
data=data.apply(zero_one6,axis=1)
data=data.apply(zero_one7,axis=1)
data=data.apply(zero_one8,axis=1)




In [16]:
data.to_csv('归一化第一版.csv')